In [1]:
import argparse
import logging
import os

import numpy as np
import torch
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
from train import train, train_and_evaluate
import matplotlib.pyplot as plt

import utils
import model.net as net
from  model.data_loader import  fetch_dataloader
from evaluate import evaluate

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Initial training

Let's start with a simple model:

- no augmentation (that is critical with such a small dataset);
- default learning rate;
- default optimizer (in the paper they use high momentum for a `batch_size = 1`); 

and so on.

In [2]:
model_dir = 'experiments/initial_model'
json_path = os.path.join(model_dir, 'params.json')
params = utils.Params(json_path)

In [3]:
params.dict

{'learning_rate': 0.001,
 'batch_size': 1,
 'num_epochs': 25,
 'save_summary_steps': 100,
 'num_workers': 0}

In [6]:
!python3 train.py --model_dir 'experiments/initial_model'

Loading the datasets...
- done.
Starting training for 25 epoch(s)
Epoch 1/25
100%|███████████████████████████████| 25/25 [00:04<00:00,  5.16it/s, loss=0.481]
- Train metrics: accuracy: 0.274 ; loss: 0.782
- Eval metrics : accuracy: 0.801 ; loss: 0.490
Checkpoint Directory exists! 
- Found new best accuracy
Epoch 2/25
100%|███████████████████████████████| 25/25 [00:02<00:00,  9.24it/s, loss=0.355]
- Train metrics: accuracy: 0.861 ; loss: 0.383
- Eval metrics : accuracy: 0.804 ; loss: 0.494
Checkpoint Directory exists! 
- Found new best accuracy
Epoch 3/25
100%|███████████████████████████████| 25/25 [00:02<00:00,  9.18it/s, loss=0.295]
- Train metrics: accuracy: 0.887 ; loss: 0.313
- Eval metrics : accuracy: 0.804 ; loss: 0.503
Checkpoint Directory exists! 
- Found new best accuracy
Epoch 4/25
100%|███████████████████████████████| 25/25 [00:02<00:00,  9.14it/s, loss=0.260]
- Train metrics: accuracy: 0.915 ; loss: 0.250
- Eval metrics : accuracy: 0.805 ; loss: 0.480
Checkpoint Directory e